In [2]:
import pandas as pd
import numpy as np
import QuantLib as ql
import datetime as dt

In [3]:
df=pd.read_csv('sample_portfolio.csv')
print(df.head)

<bound method NDFrame.head of     SecurityID   IssueDate FirstSettlementDate AccrualDate DaycountBasisType  \
0            1   5/15/2020           5/15/2020   5/15/2020     Actual/Actual   
1            2    6/1/2020            6/1/2020   5/15/2020     Actual/Actual   
2            3   7/31/2019           7/31/2019   7/31/2019     Actual/Actual   
3            4   5/15/2020           5/15/2020   5/15/2020     Actual/Actual   
4            5   7/31/2020           7/31/2020   7/31/2020     Actual/Actual   
5            7   7/31/2020           7/31/2020   7/31/2020     Actual/Actual   
6            9  11/16/2020          11/16/2020  11/15/2020     Actual/Actual   
7           10    2/1/2021            2/1/2021   1/31/2021     Actual/Actual   
8           11   2/16/2021           2/16/2021   2/15/2021     Actual/Actual   
9           12   2/16/2021           2/16/2021   2/15/2021     Actual/Actual   
10          13    3/1/2021            3/1/2021   2/15/2021     Actual/Actual   
11        

In [4]:
#Need to convert PositionNotional column to int
df[' PositionNotional ']=df[' PositionNotional '].str.replace(',', '').astype(int)

In [5]:
#drop null columns
df=df.drop(['Unnamed: 10','Unnamed: 14'],axis=1)
df.head()

,SecurityID,IssueDate,FirstSettlementDate,AccrualDate,DaycountBasisType,CouponType,Coupon,FirstCouponDate,InterestPaymentFrequency,MaturityDate,Date,Price,PositionNotional
0,1,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,1.500,11/15/2020,Semiannually,5/15/2050,10/7/2021,81.129,711000
1,2,6/1/2020,6/1/2020,5/15/2020,Actual/Actual,FIX,1.375,11/15/2020,Semiannually,5/15/2040,10/7/2021,85.852,589000
2,3,7/31/2019,7/31/2019,7/31/2019,Actual/Actual,FIX,2.000,1/31/2020,Semiannually,7/31/2024,10/7/2021,103.516,486000
3,4,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,0.875,11/15/2020,Semiannually,5/15/2030,10/7/2021,93.153,572000
4,5,7/31/2020,7/31/2020,7/31/2020,Actual/Actual,FIX,0.500,1/31/2021,Semiannually,7/31/2025,10/7/2021,98.054,887000


In [6]:
def one_bond(row):
    """
    function to process one fixed coupon bond (Actual/Actual, Semi-annual coupons)
    ---
    input: one row from data frame
    ---
    output: YTM, Modified Duration, Dirty Price, Maturity Bucket
    """

    d=dt.datetime.strptime(row['Date'], "%m/%d/%Y")
    valuationDate = ql.Date(d.day,d.month,d.year)
    ql.Settings.instance().evaluationDate = valuationDate
    
    d=dt.datetime.strptime(row['AccrualDate'], "%m/%d/%Y")
    start = ql.Date(d.day,d.month,d.year)
    
    d=dt.datetime.strptime(row['MaturityDate'], "%m/%d/%Y")
    maturity = ql.Date(d.day,d.month,d.year)
    
    #time till maturity from valuation date
    TTM = ql.ActualActual().yearFraction(valuationDate, maturity)
    if TTM <= 5:
        mat_bucket = '5Y'
    elif TTM < 10:
        mat_bucket = '10Y'
    elif TTM < 20:
        mat_bucket = '20Y'
    elif TTM < 30:
        mat_bucket = '30Y'
        
    calendar = ql.UnitedStates()
    frequency = ql.Semiannual
    tenor = ql.Period(frequency)
    dateGeneration = ql.DateGeneration.Backward
    monthEnd = True if d.day in [28,29,30,31] else False
    schedule = ql.Schedule (start, maturity, tenor, calendar, ql.Unadjusted, ql.Unadjusted, dateGeneration, monthEnd)
    daycount = ql.ActualActual(ql.ActualActual.ISMA,schedule)
    
    settlementDays = 0
    price=row['Price']
    face=100
    coupon = row['Coupon']/100
    
    bond = ql.FixedRateBond(settlementDays, face, schedule, [coupon], daycount)
    ytm = bond.bondYield(price, daycount, ql.Compounded, frequency,)
    
    rate = ql.InterestRate(ytm, daycount, ql.Compounded, frequency)
    ModDur = ql.BondFunctions.duration(bond,rate,ql.Duration.Modified)
    dirty_p = bond.dirtyPrice(ytm, daycount, ql.Compounded, frequency)
    
    return pd.Series([ytm, ModDur, dirty_p,mat_bucket])



In [7]:
#apply function to calculate YTM, duration, market price and maturity bucket for all bonds in portfolio
newdf=df.apply(one_bond, axis=1)
newdf.rename(columns={0: "YTM", 1: "Duration",2:"DirtyPrice",3:'Maturity_Bucket'},inplace=True)
newdf.head()

,YTM,Duration,DirtyPrice,Maturity_Bucket
0,0.024176,22.214099,81.720024,30Y
1,0.023151,15.978642,86.393785,20Y
2,0.007356,2.733512,103.885564,5Y
3,0.017349,8.193545,93.497766,10Y
4,0.010213,3.760832,98.146392,5Y


In [8]:
df=pd.concat([df,newdf],axis=1)
df.head()

,SecurityID,IssueDate,FirstSettlementDate,AccrualDate,DaycountBasisType,CouponType,Coupon,FirstCouponDate,InterestPaymentFrequency,MaturityDate,Date,Price,PositionNotional,YTM,Duration,DirtyPrice,Maturity_Bucket
0,1,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,1.500,11/15/2020,Semiannually,5/15/2050,10/7/2021,81.129,711000,0.024176,22.214099,81.720024,30Y
1,2,6/1/2020,6/1/2020,5/15/2020,Actual/Actual,FIX,1.375,11/15/2020,Semiannually,5/15/2040,10/7/2021,85.852,589000,0.023151,15.978642,86.393785,20Y
2,3,7/31/2019,7/31/2019,7/31/2019,Actual/Actual,FIX,2.000,1/31/2020,Semiannually,7/31/2024,10/7/2021,103.516,486000,0.007356,2.733512,103.885564,5Y
3,4,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,0.875,11/15/2020,Semiannually,5/15/2030,10/7/2021,93.153,572000,0.017349,8.193545,93.497766,10Y
4,5,7/31/2020,7/31/2020,7/31/2020,Actual/Actual,FIX,0.500,1/31/2021,Semiannually,7/31/2025,10/7/2021,98.054,887000,0.010213,3.760832,98.146392,5Y


In [9]:
#DV01=Modified Duration*0.01 * MarketPrice*0.01
df['DV01']=df['DirtyPrice']/100*df[' PositionNotional ']*df['Duration']*0.0001

In [10]:
#check and verify day count between any two semi-annual coupon payments is 184 
ql.ActualActual().dayCount(ql.Date(15, 5, 2021), ql.Date(15, 11, 2021))

184

In [11]:
#number of days from 10/7/2021 to 10/12/2021
ndays=ql.Date(12, 10, 2021)-ql.Date(7, 10, 2021)
ndays

5

In [12]:
#Calculate Accrued Interest on 10/7/2021
AccruedInterest_10_07=(df['DirtyPrice'] - df['Price'])/100 * df[' PositionNotional ']

#Accrued Interest on 10/12/2021
df['AccruedInterest_10.12']=AccruedInterest_10_07 + df['Coupon']/100/2 * df[' PositionNotional '] * ndays/184

In [13]:
df.head()

,SecurityID,IssueDate,FirstSettlementDate,AccrualDate,DaycountBasisType,CouponType,Coupon,FirstCouponDate,InterestPaymentFrequency,MaturityDate,Date,Price,PositionNotional,YTM,Duration,DirtyPrice,Maturity_Bucket,DV01,AccruedInterest_10.12
0,1,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,1.500,11/15/2020,Semiannually,5/15/2050,10/7/2021,81.129,711000,0.024176,22.214099,81.720024,30Y,1290.704425,4347.087539
1,2,6/1/2020,6/1/2020,5/15/2020,Actual/Actual,FIX,1.375,11/15/2020,Semiannually,5/15/2040,10/7/2021,85.852,589000,0.023151,15.978642,86.393785,20Y,813.088196,3301.149274
2,3,7/31/2019,7/31/2019,7/31/2019,Actual/Actual,FIX,2.000,1/31/2020,Semiannually,7/31/2024,10/7/2021,103.516,486000,0.007356,2.733512,103.885564,5Y,138.010626,1928.145972
3,4,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,0.875,11/15/2020,Semiannually,5/15/2030,10/7/2021,93.153,572000,0.017349,8.193545,93.497766,10Y,438.196731,2040.066030
4,5,7/31/2020,7/31/2020,7/31/2020,Actual/Actual,FIX,0.500,1/31/2021,Semiannually,7/31/2025,10/7/2021,98.054,887000,0.010213,3.760832,98.146392,5Y,327.402389,879.778825


In [15]:
#2.Aggregate DV01,PositionNotional,Accrued Interest on the maturity buckets
aggdf = df.groupby('Maturity_Bucket').agg({'DV01':'sum',' PositionNotional ':'sum','AccruedInterest_10.12':'sum'})
aggdf

,DV01,PositionNotional,AccruedInterest_10.12
Maturity_Bucket,,,
10Y,6910.999276,9717000,24605.547345
20Y,4138.454593,2676000,15042.229789
30Y,2809.330183,1389000,8395.561530
5Y,4364.430050,11547000,13386.157966


In [16]:
#3.case testing
port_dv01 = aggdf['DV01'].sum()
port_dv01

18223.214101413803

In [17]:
test = np.array([-25,-20,-15,-10,-5,5,10,15,20,25])
p_and_l = np.round(-test*port_dv01,2)
cols=[str(bps)+'bps' for bps in test]
pd.DataFrame([p_and_l],columns=cols,index=['Portfolio P&L'] )

,-25bps,-20bps,-15bps,-10bps,-5bps,5bps,10bps,15bps,20bps,25bps
Portfolio P&L,455580.35,364464.28,273348.21,182232.14,91116.07,-91116.07,-182232.14,-273348.21,-364464.28,-455580.35


In [18]:
def cal_p_and_l(data):
    """
    input:a row from dataframe
    output: p&l
    """
    dv01 = data['DV01']
    test = np.array([-25,-20,-15,-10,-5,5,10,15,20,25])
    p_and_l = -test*dv01
    cols=[str(bps)+'bps' for bps in test]
    
    return pd.Series(p_and_l,index=cols)

In [19]:
bucket_p_L = aggdf.apply(cal_p_and_l, axis=1)
bucket_p_L.loc['Portfolio Total']=bucket_p_L.sum(axis=0)
np.round(bucket_p_L,2)

,-25bps,-20bps,-15bps,-10bps,-5bps,5bps,10bps,15bps,20bps,25bps
Maturity_Bucket,,,,,,,,,,
10Y,172774.98,138219.99,103664.99,69109.99,34555.00,-34555.00,-69109.99,-103664.99,-138219.99,-172774.98
20Y,103461.36,82769.09,62076.82,41384.55,20692.27,-20692.27,-41384.55,-62076.82,-82769.09,-103461.36
30Y,70233.25,56186.60,42139.95,28093.30,14046.65,-14046.65,-28093.30,-42139.95,-56186.60,-70233.25
5Y,109110.75,87288.60,65466.45,43644.30,21822.15,-21822.15,-43644.30,-65466.45,-87288.60,-109110.75
Portfolio Total,455580.35,364464.28,273348.21,182232.14,91116.07,-91116.07,-182232.14,-273348.21,-364464.28,-455580.35
